In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
import os
import joblib
from sklearn.svm import SVC

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
df = pd.read_csv(r"cyberbullying_tweets.csv")

In [ ]:
df

In [ ]:
target_class_size = 500

# Undersample each class
undersampled_data = df.groupby('cyberbullying_type').apply(lambda x: x.sample(n=target_class_size, random_state=42) if len(x) >= target_class_size else x)

# Reset index to clean up the DataFrame after grouping
undersampled_data.reset_index(drop=True, inplace=True)

# Display the shape of the undersampled data
print(f"Shape of the undersampled data: {undersampled_data.shape}")

# Optionally, display the class distribution in the undersampled data
class_distribution = undersampled_data['cyberbullying_type'].value_counts()
print("Class distribution in the undersampled data:")
print(class_distribution)

In [ ]:
# Drop all rows with the class name 'gender'
undersampled_data = undersampled_data[undersampled_data['cyberbullying_type'] != 'gender']
undersampled_data = undersampled_data[undersampled_data['cyberbullying_type'] != 'not_cyberbullying']

# Optionally, reset the index again if needed
undersampled_data.reset_index(drop=True, inplace=True)

# Display the shape of the updated undersampled data
print(f"Shape of the undersampled data after dropping 'gender': {undersampled_data.shape}")

# Optionally, display the class distribution in the updated undersampled data
class_distribution = undersampled_data['cyberbullying_type'].value_counts()
print("Class distribution in the updated undersampled data:")
print(class_distribution)

In [ ]:
df=undersampled_data

In [ ]:
df

In [ ]:
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define the NLP preprocessing function
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming
    tokens = [stemmer.stem(word) for word in tokens]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word, pos='v') for word in tokens]  # pos='v' for verb lemmatization

    return " ".join(tokens)

In [ ]:
df['preprocessed_text'] = df['tweet_text'].apply(preprocess_text)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(df['preprocessed_text'])

In [ ]:
X, y = tfidf_features, df['cyberbullying_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 41)

In [ ]:
#defining global variables to store accuracy and other metrics
precision = []
recall = []
fscore = []
accuracy = []

In [2]:
#function to calculate various metrics such as accuracy, precision etc
def calculateMetrics(algorithm, testY,predict):

    p = precision_score(testY, predict,average='macro') * 100
    r = recall_score(testY, predict,average='macro') * 100
    f = f1_score(testY, predict,average='macro') * 100
    a = accuracy_score(testY,predict)*100 
    accuracy.append(a)
    precision.append(p)
    recall.append(r)
    fscore.append(f)
    print(algorithm+' Accuracy    : '+str(a))
    print(algorithm+' Precision   : '+str(p))
    print(algorithm+' Recall      : '+str(r))
    print(algorithm+' FSCORE      : '+str(f))
    report=classification_report(predict, testY,target_names=labels)
    print('\n',algorithm+" classification report\n",report)
    conf_matrix = confusion_matrix(testY, predict) 
    plt.figure(figsize =(5, 5)) 
    ax = sns.heatmap(conf_matrix, xticklabels = labels, yticklabels = labels, annot = True, cmap="Blues" ,fmt ="g");
    ax.set_ylim([0,len(labels)])
    plt.title(algorithm+" Confusion matrix") 
    plt.ylabel('True class') 
    plt.xlabel('Predicted class') 
    plt.show()

In [3]:
labels=set(df['cyberbullying_type'])

NameError: name 'df' is not defined

In [4]:
if os.path.exists('model/svm.pkl'):
    # Load the trained model from the file
    svc = joblib.load('model/svm.pkl')
    print("Model loaded successfully.")
    predict = svc.predict(X_test)
    calculateMetrics("svm", predict, y_test)
else:
    # Train the model (assuming X_train and y_train are defi}ned)
    svc = SVC(kernel='linear')  # You can also use 'rbf', 'poly', etc.
    svc.fit(X_train, y_train)
    # Save the trained model to a file
    joblib.dump(svc, 'model/svm.pkl')
    print("Model saved successfully.")
    predict = svc.predict(X_test)
    calculateMetrics("svm", predict, y_test)

NameError: name 'os' is not defined

In [5]:
if os.path.exists('model/RFC2.pkl'):
    # Load the trained model from the file
    rfc = joblib.load('model/RFC2.pkl')
    print("Model loaded successfully.")
    predict = rfc.predict(X_test)
    calculateMetrics("RFC", predict, y_test)
else:
    # Train the model (assuming X_train and y_train are defi}ned)
    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    # Save the trained model to a file
    joblib.dump(rfc, 'model/RFC2.pkl')
    print("Model saved successfully.")
    predict = rfc.predict(X_test)
    calculateMetrics("RFC", predict, y_test)

NameError: name 'os' is not defined

In [6]:
# Load the test data
test_df = pd.read_csv('test.csv')

# Define the preprocessing function
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    tokens = [lemmatizer.lemmatize(word, pos='v') for word in tokens]  # pos='v' for verb lemmatization
    return " ".join(tokens)

# Preprocess the test tweets
test_df['preprocessed_text'] = test_df['tweet_text'].apply(preprocess_text)

# Vectorize the preprocessed test data using the loaded TF-IDF vectorizer
test_features = tfidf_vectorizer.transform(test_df['preprocessed_text'])

# Make predictions on the test data
predictions = rfc.predict(test_features)

# Add predictions to the test DataFrame
test_df['predicted_cyberbullying_type'] = predictions


NameError: name 'stopwords' is not defined

In [7]:
test_df

tweet_text   cyberbullying_type  \
0   I am watching a documentary called I am not yo...            ethnicity   
1   Dude you’re an idiot. It makes sense to be pre...             religion   
2   @aquarianfool when i try to talk about, it's o...  other_cyberbullying   
3              @Skawtnyc @gregtitus  that's a relief.  other_cyberbullying   
4       OMG SHUT UP DRASKO AND BIANCA #MKR #FINALFIVE  other_cyberbullying   
5   You really are quoting a guy who promised to p...             religion   
6   RT ignorance... “@tayyoung_: FUCK OBAMA, dumb ...            ethnicity   
7   Filthy Rich Boys: A Reverse Harem High School ...                  age   
8   RT @yjoyding: Followed by "There are a couple ...  other_cyberbullying   
9   @seijohgorl hoy mami keito AAAA okay so um ika...            ethnicity   
10  @beardyblue @emily_esque shoot me an email, ra...  other_cyberbullying   
11  RT @shanley: They then start digging in detail...  other_cyberbullying   
12  You are an extremist who gets to label every M...             religion   
13  RT @Lauraka09619649: I bet #Tucker takes an in...  other_cyberbullying   
14  the same people that say galentine’s day uniro...                  age   
15  She isn’t responsible to Delhi riots, who is r...             religion   
16  Black people: I get called nigger, gorilla, co...            ethnicity   
17  My take on twitter's safety update: propaganda...  other_cyberbullying   
18  You dumb fuck thats how you spell it you stupi...            ethnicity   
19  @darth_schmoo if i ever have my own wikipedia ...  other_cyberbullying   
20  These girls who were a year older and bullied ...                  age   
21  Bobby has made many tweets in support of the L...             religion   
22  I agree, there are always people that take thi...             religion   
23  Cause I hate to break it to them, I'm gettin c...            ethnicity   
24  This for those who say "negro" is patently off...            ethnicity   
25  #NakagawaTaishi and #IshiiAnna to be lead in m...                  age   
26  @1fatoshn Tell it to the 120 million Africans ...             religion   
27  Why do people loved to bully others? Yeah it d...                  age   
28  niggers are fucking STUPID I hate these fuck b...            ethnicity   
29  A girl who bullied me in high school reached o...                  age   
30  Ngl my Muslim friends are the kindest and most...             religion   
31  Process of illumination, Nimrod, was a Hammite...            ethnicity   
32  Imam's who disagree with Jihad are attacked by...             religion   
33  turn yo location on on yo tweets homie lmao “@...            ethnicity   
34  RT @Rapmonster: fuck some gucci.. niggers is D...            ethnicity   
35  @i_am_tyj. Fuck you #niggerlover she's still a...            ethnicity   
36  Caste is like race.. u marry a white american ...            ethnicity   
37  In jr high a girl “bullied” me and spread rumo...                  age   
38  Oh damn I thought you were siding with me. The...            ethnicity   
39  You don't believe in heaven or hell It's not m...             religion   
40  Know who it does help? The family of the victi...                  age   
41  This girl who looked like a man bullied the fu...                  age   
42  Sleepytime peach tea with lavender infused hon...  other_cyberbullying   
43  @stevie_reed your just bullying me, you love m...  other_cyberbullying   
44                          @_Vanna817 who bullying u  other_cyberbullying   
45  @MaxBlumenthal Islamophobia - fear of this. ht...             religion   
46  Soon enough her Shadowy Dress wrapped over bot...            ethnicity   
47  Worm of Islamic militant http://jihad.my langu...             religion   
48  Don't be surprised.Every terrorist act that IS...             religion   
49  The fact that you added ji to Gandhi tells me ...             religion   

                                    preprocessed_text  
0   watc